# Task 1 SQL

Load libraries and packages

In [3]:
#import libraries
import sqlite3
import pandas as pd

Data ingestion into notebook

In [5]:
#load csv files
esg_metrics = pd.read_csv('/content/OEGEN/data_files/esg_metrics.csv')
fund_names = pd.read_csv('/content/OEGEN/data_files/fund_names.csv')
mod_pro_data = pd.read_csv('/content/OEGEN/data_files/modified_production_data_subset.csv')
site_details = pd.read_csv('/content/OEGEN/data_files/site_details.csv')
site_fund_os_perc = pd.read_csv('/content/OEGEN/data_files/site_fund_ownership_percentage.csv')

Below script analyses data to check for nulls, inspect data types and inspect columns

In [ ]:
def info(df):
  return_info = df.info()
  first_five = df.head()
  return_null = df.isnull().sum()
  return return_info, first_five, return_null

print(info(esg_metrics))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1032 entries, 0 to 1031
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   metric_id     1032 non-null   object        
 1   metric_name   1032 non-null   object        
 2   technology    1032 non-null   object        
 3   country       1032 non-null   object        
 4   metric_value  1032 non-null   float64       
 5   metric_unit   1032 non-null   object        
 6   source        1032 non-null   object        
 7   created_at    1032 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(6)
memory usage: 64.6+ KB
(None,   metric_id                           metric_name technology    country  \
0      ESG1  Average Household Energy Consumption    Unknown  Australia   
1      ESG2  Average Household Energy Consumption    Unknown    Austria   
2      ESG3  Average Household Energy Consumption    Unknown    Belgium   
3      ESG4  Av

### Data overview
esg_metrics type/ null count:
*   metric_id: object/ 0
*   metric_name: object/ 0
*   technology: object/ 136
*   country: object/ 8
*   metric_value: object/ 0
*   metric_unit: object/ 0
*   source: object/ 0
*   created_at: object/ 0


fund_names type/ null count:
*   fund_id: int/ 0
*   fund_name: object/ 0

mod_pro_data type/ null count:
*   site_id: float64/ 1905
*   report_date: object/ 0
*   actual_production: float64/ 29

site_details type/ null count:
*   site_id: int/ 0
*   technology: object/ 0
*   country: object/ 0
*   latitude: float/ 43
*   longitude: float/ 43
*   asset_phase: object/ 21

site_fund_os_perc type/ null count:
*   site_id: int/ 0
*   fund_id: int/ 0
*   fund_ownership: float/ 0

## Data cleaning and processing

In [6]:
#clean null values and correct data types
#esg_metrics table
esg_metrics['technology'].fillna('Unknown', inplace=True)
esg_metrics['country'].fillna('Unknown', inplace=True)
esg_metrics['created_at'] = pd.to_datetime(esg_metrics['created_at'])

#mod_pro_data
#drop null
mod_pro_data.dropna(subset=['site_id'], inplace=True)
#change data type to string
mod_pro_data['site_id'] = mod_pro_data['site_id'].astype(int).astype(str)
#change data type to date
mod_pro_data['report_date'] = pd.to_datetime(mod_pro_data['report_date'])
#new column of average actual production per site id
mod_pro_data['avg_ap'] = mod_pro_data.groupby([mod_pro_data['site_id'], mod_pro_data['report_date'].dt.year])['actual_production'].transform('mean')
#fill empty cells for actuall production
mod_pro_data['actual_production'].fillna(mod_pro_data['avg_ap'], inplace=True)
#fill any existing empty entries with 0
mod_pro_data['actual_production'].fillna(0, inplace=True)
#drop the avg_ap column
mod_pro_data.drop(columns=['avg_ap'],inplace=True)

#site_details
#fill missing lat and long values
site_details.sort_values(by='country', inplace=True)
#create function to fill empty cells by country (approx lat and long)
def fill_miss(group):
  group['latitude'] = group['latitude'].fillna(method='ffill')
  group['longitude'] = group['longitude'].fillna(method='ffill')
  return group
#fill lat and long by country
site_details = site_details.groupby('country').apply(fill_miss)
#reset index from groupby
site_details.reset_index(drop=True, inplace=True)
#describe asset phase for site details
site_details['asset_phase'].fillna('Unknown', inplace=True)
#change site_id data type to string
site_details['site_id'] = site_details['site_id'].astype(str)

#site_fund_os_perc
#change site_id and fund_id to string
site_fund_os_perc['site_id'] = site_fund_os_perc['site_id'].astype(str)
site_fund_os_perc['fund_id'] = site_fund_os_perc['fund_id'].astype(str)

#fund_names
#change fund_id to string
fund_names['fund_id'] = fund_names['fund_id'].astype(str)

<ipython-input-6-500b0c921027>:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  esg_metrics['created_at'] = pd.to_datetime(esg_metrics['created_at'])


## Convert dataframe to SQL tables

In [7]:
#create connection to sqlite db
conn = sqlite3.connect('esg_database.db')

esg_metrics.to_sql('esg_metrics_sql', conn, if_exists = 'replace', index=False)
fund_names.to_sql('fund_names_sql', conn, if_exists = 'replace', index=False)
mod_pro_data.to_sql('mod_pro_data_sql', conn, if_exists = 'replace', index=False)
site_details.to_sql('site_details_sql', conn, if_exists = 'replace', index=False)
site_fund_os_perc.to_sql('site_fund_os_perc_sql', conn, if_exists = 'replace', index=False)

conn.close()

Test database connection with simple query

In [ ]:
conn = sqlite3.connect('esg_database.db')

query = '''
SELECT *
FROM esg_metrics_sql
LIMIT 10
'''
df=pd.read_sql(query,conn)
print(df)

  metric_id                           metric_name technology         country  \
0      ESG1  Average Household Energy Consumption    Unknown       Australia   
1      ESG2  Average Household Energy Consumption    Unknown         Austria   
2      ESG3  Average Household Energy Consumption    Unknown         Belgium   
3      ESG4  Average Household Energy Consumption    Unknown        Bulgaria   
4      ESG5  Average Household Energy Consumption    Unknown         Croatia   
5      ESG6  Average Household Energy Consumption    Unknown          Cyprus   
6      ESG7  Average Household Energy Consumption    Unknown  Czech Republic   
7      ESG8  Average Household Energy Consumption    Unknown         Denmark   
8      ESG9  Average Household Energy Consumption    Unknown         Estonia   
9     ESG10  Average Household Energy Consumption    Unknown         Finland   

   metric_value metric_unit   source created_at  
0         5.825         MWh     AEMC    00:31.2  
1         5.046    

Create single table for ESG reference metrics and question 2 answer

In [9]:
conn = sqlite3.connect('esg_database.db')

# Create the table with the results from the query
create_table_query = '''
CREATE TABLE esg_metrics_table AS
SELECT
    sd.site_id,
    sd.technology,
    sd.country,
    fn.fund_name,
    po.report_date,
    po.scaled_production AS actual_production,
    em.carbon_avoided_gco2_kwh,
    (po.scaled_production * (em.carbon_avoided_gco2_kwh / 1000)) AS tonnes_carbon_avoided,
    sd.latitude,
    sd.longitude
FROM
    site_details_sql sd
JOIN
    (
        SELECT
            pd.site_id,
            pd.report_date,
            pd.actual_production * os.fund_ownership AS scaled_production,
            os.fund_id
        FROM
            mod_pro_data_sql pd
        JOIN
            site_fund_os_perc_sql os ON pd.site_id = os.site_id
    ) po ON sd.site_id = po.site_id
JOIN
    (
        SELECT
            em.technology,
            em.country,
            em.metric_value AS carbon_avoided_gco2_kwh,
            em.created_at
        FROM
            esg_metrics_sql em
        JOIN
            (
                SELECT
                    technology,
                    country,
                    MAX(created_at) as latest_date
                FROM
                    esg_metrics_sql
                WHERE
                    metric_name = 'Carbon Avoided'
                GROUP BY
                    technology, country
            ) lem ON em.technology = lem.technology
                   AND em.country = lem.country
                   AND em.created_at = lem.latest_date
    ) em ON (sd.technology = em.technology OR em.technology IS NULL)
          AND (sd.country = em.country OR em.country IS NULL)
JOIN
    fund_names_sql fn ON po.fund_id = fn.fund_id
'''

# Execute the query to create the table
conn.execute(create_table_query)
conn.commit()

# Verify the table creation by fetching the first few rows
#df = pd.read_sql('SELECT * FROM esg_metrics_table LIMIT 5', conn)
#print(df.head())

create_avg_offset_grnd_mnt_slr = '''
CREATE TABLE avg_offset_ground_mount_solar AS
SELECT
    fund_name,
    FinalESGTable.Technology,
    AVG(tonnes_carbon_avoided) AS average_tonnes_carbon_avoided
FROM
    (
        SELECT
            sd.site_id,
            sd.technology,
            sd.country,
            fn.fund_name,
            po.report_date,
            po.scaled_production,
            em.carbon_avoided_gco2_kwh,
            (po.scaled_production * (em.carbon_avoided_gco2_kwh / 1000)) AS tonnes_carbon_avoided
        FROM
            site_details_sql sd
        JOIN
            (
                SELECT
                    pd.site_id,
                    pd.report_date,
                    pd.actual_production * os.fund_ownership AS scaled_production,
                    os.fund_id
                FROM
                    mod_pro_data_sql pd
                JOIN
                    site_fund_os_perc_sql os
                ON
                    pd.site_id = os.site_id
            ) po
        ON
            sd.site_id = po.site_id
        JOIN
            (
                SELECT
                    em.technology,
                    em.country,
                    em.metric_value AS carbon_avoided_gco2_kwh,
                    em.created_at
                FROM
                    esg_metrics_sql em
                JOIN
                    (
                        SELECT
                            em_sub.technology,
                            em_sub.country,
                            MAX(em_sub.created_at) as latest_date
                        FROM
                            esg_metrics_sql em_sub
                        WHERE
                            em_sub.metric_name = 'Carbon Avoided'
                        GROUP BY
                            em_sub.technology, em_sub.country
                    ) lem
                ON
                    em.technology = lem.technology
                    AND em.country = lem.country
                    AND em.created_at = lem.latest_date
            ) em
        ON
            (sd.technology = em.technology OR em.technology IS NULL)
            AND (sd.Country = em.country OR em.country IS NULL)
        JOIN
            fund_names_sql fn
        ON
            po.fund_id = fn.fund_id
        WHERE
            sd.technology = 'Ground mount solar'
            AND po.Report_date BETWEEN '2023-01-01' AND '2023-03-31'
    ) FinalESGTable
GROUP BY
    fund_name,
    FinalESGTable.Technology
'''

#Execute the query to create the table
conn.execute(create_avg_offset_grnd_mnt_slr)
conn.commit()

#Verify the table creation by fetching the first few rows, testing
#df_avg = pd.read_sql('SELECT * FROM avg_offset_ground_mount_solar' LIMIT 5, conn)
#print(df_avg.head())

#Close connection
conn.close()

OperationalError: table esg_metrics_table already exists

## Question 1 - Task 1

In [10]:
conn = sqlite3.connect('esg_database.db')

#Answer
df_avg = pd.read_sql('SELECT * FROM avg_offset_ground_mount_solar', conn)
print(df_avg)

conn.close()

    fund_name          technology  average_tonnes_carbon_avoided
0      Banana  Ground mount solar                     380.216831
1       Grape  Ground mount solar                   30173.185169
2  Indian Fig  Ground mount solar                     264.945957
3   Jackfruit  Ground mount solar                     293.562619
4        Kiwi  Ground mount solar                     225.762768
5       Lemon  Ground mount solar                     210.146379


## Export SQL Database to CSV for use in Power BI

In [ ]:
#SQL database to CSV
conn = sqlite3.connect('esg_database.db')

esg_csv = pd.read_sql('SELECT * FROM esg_metrics_table', conn)
esg_csv.to_csv('esg_metrics_final.csv', index=False)

conn.close()

## Task 2 Power BI

**Key measures, columns and manipulations**
*   Measure: Average Carbon avoided per Site -
this measure displays the total carbon avoided divided by the number of sites
*   Mesure: Total tonne carbon avoided - sums all total carbon avoided across all sites
*   Column: PredictedTonnesCarbonAvoided - prediction for future values that were '0' for tonne carbon avoided
*   Column: MovingAvgTCA - using moving average to predict tonne carbon avoided values for future report dates


**Uncertainties to raise and gather feedback on**
*   Unsure which metrics would be used for *ESG Carbon Metrics*
*   Missing values for technology and country in ESG Metrics data
*   Zero values in future values of modified production data

**Maintenance Considerations**
*   Establishing automated data pipeline so that up-to-date production and ESG data is displayed within dashboard
*   Investigate the causes behind null values or '0's and find ways to fill these values
*   Running data connection tests to ensure the process runs smoothly, and when changes are made to data pipelines or dashboard
*   Discuss with stakeholders the requirements for data
 visualisation and investigate particular data points in more depth, with these changes being reflected in future dashboard iterations
*   Documentation, document what data is ingested and how, document the purpose of each visual and provide details on how to use the dashboard, provide further support with training sessions for end-users
*   Version control: Maintain documentation of changes to the dashboard and track current and future changes through Jira (or any other SDLC tracking tools)







# Task 3 API

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time

#Define the endpoint
api_endpoint = "https://data.elexon.co.uk/bmrs/api/v1/generation/outturn"

#Function to test API connection
def test_api_connection():
    try:
        response = requests.get(api_endpoint)
        if response.status_code == 200:
            print("Successfully connected to the API.")
            print("Sample response data:", response.json())
            return True
        else:
            print(f"Failed to connect to the API. Status code: {response.status_code}")
            print("Response content:", response.content)
            return False
    except Exception as e:
        print(f"An error occurred: {e}")
        return False

#Function to fetch data from the API
def fetch_data(start_date, end_date):
    params = {
        "startTime": start_date,
        "endTime": end_date,
        "limit": 1000
    }

    response = requests.get(api_endpoint, params=params)
    response.raise_for_status()
    return response.json()

#Function to get data between two dates
def get_data_between_dates(start_date, end_date):
    current_date = start_date
    all_data = []

    while current_date < end_date:
        next_date = min(current_date + timedelta(days=6), end_date)
        print(f"Fetching data from {current_date} to {next_date - timedelta(seconds=1)}")

        try:
            data = fetch_data(current_date.strftime("%Y-%m-%dT%H:%M:%SZ"), (next_date - timedelta(seconds=1)).strftime("%Y-%m-%dT%H:%M:%SZ"))
            all_data.extend(data['data'])
        except Exception as e:
            print(f"Failed to fetch data: {e}")

        current_date = next_date
        time.sleep(1)  #Delay to avoid rate limiting

    return pd.DataFrame(all_data)

#Define the date range
start_date = datetime(2024, 1, 1)
end_date = datetime(2024, 3, 31)

#Check API connection before fetching data
if test_api_connection():
    #Get data
    data = get_data_between_dates(start_date, end_date)

    #Check if data retrieval was successful
    if not data.empty:
        #Convert to DataFrame
        df = pd.DataFrame(data)

        # rint the columns to debug
        print("Columns in DataFrame:", df.columns)
        print("Sample data:", df.head())

        #Remove potential duplicates
        df.drop_duplicates(subset=['startTime'], inplace=True)

        #Convert 'startTime' to datetime
        df['startTime'] = pd.to_datetime(df['startTime'])

        #Sum the demand values for each timestamp
        total_generation_outturn = df.groupby('startTime')['demand'].sum().reset_index()

        #Output the DataFrame
        print("DataFrame of total generation outturn:")
        print(total_generation_outturn)

        #Calculate the total sum of generation outturn for the period
        total_outturn_sum = total_generation_outturn['demand'].sum()
        print(f"Total Generation Outturn from {start_date.date()} to {end_date.date()}: {total_outturn_sum}")
    else:
        print("No data retrieved. Please check the API endpoint and parameters.")


Successfully connected to the API.
Sample response data: {'metadata': {'datasets': ['FUELINST']}, 'data': [{'recordType': 'VD', 'startTime': '2024-07-02T01:30:00Z', 'demand': 23151}, {'recordType': 'VD', 'startTime': '2024-07-02T01:35:00Z', 'demand': 23033}, {'recordType': 'VD', 'startTime': '2024-07-02T01:40:00Z', 'demand': 22908}, {'recordType': 'VD', 'startTime': '2024-07-02T01:45:00Z', 'demand': 22847}, {'recordType': 'VD', 'startTime': '2024-07-02T01:50:00Z', 'demand': 22740}, {'recordType': 'VD', 'startTime': '2024-07-02T01:55:00Z', 'demand': 22856}, {'recordType': 'VD', 'startTime': '2024-07-02T02:00:00Z', 'demand': 23053}, {'recordType': 'VD', 'startTime': '2024-07-02T02:05:00Z', 'demand': 23087}, {'recordType': 'VD', 'startTime': '2024-07-02T02:10:00Z', 'demand': 23071}, {'recordType': 'VD', 'startTime': '2024-07-02T02:15:00Z', 'demand': 23040}, {'recordType': 'VD', 'startTime': '2024-07-02T02:20:00Z', 'demand': 23026}, {'recordType': 'VD', 'startTime': '2024-07-02T02:25:00Z',